In [57]:
# 0 = no streamlit
# 1 = test user inputs
# 2 = run in streamlit
streamlit_status = 0
#file_name_pickle_read = 'model_2020_06_06_1105.pickle'
do_plots = 1

dir_read = '/Users/rachellehorwitz/Documents/ViTalErt/data/filtered05/'
#dir_read = '/Users/rachellehorwitz/Documents/VTAlert/over18_eicu/'

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, confusion_matrix, plot_confusion_matrix, f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, learning_curve, train_test_split, ShuffleSplit, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import streamlit as st
import pickle
from datetime import datetime
import pyprojroot


matplotlib.rcParams.update({'font.size': 22})

# Function Definitions

In [58]:
def now_to_str():
    now = str(datetime.now())
    return now[0:4] + '_' + now[5:7] + '_' + now[8:10] + '_' + now[11:13] + now[14:16]

# Defining X and y

In [59]:
Xy = pd.read_csv(dir_read + 'Xy_2020_06_12_1524.csv')
Xy = Xy.set_index('patientunitstayid')
y = Xy.pop('label')
X = Xy.copy()
X.columns.shape

(30,)

In [60]:
X.head()

,age,admissionweight,admissionheight,bmi,gender_Female,ethnicity_African American,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,...,immunosuppression,cirrhosis,activetx,ima,midur,oobventday1,oobintubday1,diabetes,visitnumber,heartrate
patientunitstayid,,,,,,,,,,,,,,,,,,,,,
141168,70,84.3,152.4,36.295906,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,125.052830
141194,68,73.9,180.3,22.732803,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,86.860627
141197,71,102.1,162.6,38.617545,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,97.307692
141203,77,70.2,160.0,27.421875,1,0,0,1,0,0,...,0,0,1,0,0,1,0,1,1,91.543554
141208,25,95.3,172.7,31.952749,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,77.817460


In [61]:
cols = list(X.columns)

In [62]:
cols

['age',
 'admissionweight',
 'admissionheight',
 'bmi',
 'gender_Female',
 'ethnicity_African American',
 'ethnicity_Asian',
 'ethnicity_Caucasian',
 'ethnicity_Hispanic',
 'ethnicity_Native American',
 'ethnicity_Other/Unknown',
 'verbal',
 'motor',
 'eyes',
 'thrombolytics',
 'aids',
 'hepaticfailure',
 'lymphoma',
 'metastaticcancer',
 'leukemia',
 'immunosuppression',
 'cirrhosis',
 'activetx',
 'ima',
 'midur',
 'oobventday1',
 'oobintubday1',
 'diabetes',
 'visitnumber',
 'heartrate']

In [63]:
vars_categ = ['gender_Female', 'ethnicity_AfricanAmerican', 'ethnicity_Asian', 'ethnicity_Caucasian', \
             'ethnicity_Hispanic', 'ethnicity_NativeAmerican', 'ethnicity_Other/Unknown',\
              'unitstaytype_admit', 'unitstaytype_readmit', 'unitstaytype_transfer', \
             'thrombolytics', 'aids', 'hepaticfailure' 'lymphoma', 'metastiticcancer', 'leukemia', \
             'immunosuppression', 'cirrhosis', 'activetx', 'ima', 'midur',
               'ventday1', 'oobventday1', 'oobintubday1', 'diabetes']

In [64]:
vars_cont = ['age', 'admissionweight', 'admissionheight', 'bmi', \
             'verbal', 'motor', 'eyes', 'visitnumber', 'heartrate']

# Logistic Regression with Polynomial Features

In [65]:
def grid_search_wrapper(clf, X_train_sc, y_train, scoring):#, X_test_sc, y_test, scoring):
    """
    fits a GridSearchCV classifier using refit_score for optimization
    prints classifier performance metrics
    """
    skf = StratifiedKFold(n_splits=5)    
    
    weights = np.array([0.001, \
                        y[y==1].shape[0] / (y[y==1].shape[0] + y.shape[0]), \
                        0.005, 0.01, 0.2, 0.5])
    
    hyperparam_grid = {'penalty': ['l1', 'l2'], \
                       'C': np.logspace(-3, 3, 5), \
                       #'class_weight': [{0: x,   1: 1-x} for x in weights], \
                       'fit_intercept': [True, False]}
    #GridSearchCV(clf, param, scoring='f1_score',refit=True,cv=10)
    
    
    grid_search = GridSearchCV(clf, scoring=scoring, param_grid=hyperparam_grid, refit=True,
                           cv=skf, return_train_score=True, n_jobs=4)
    grid_search.fit(X_train_sc, y_train)

    # make the predictions
    #y_pred = grid_search.predict(X_test_sc)

    print('Best params for ' + scoring)
    print(grid_search.best_params_)

#     # confusion matrix on the test data.
#     print('\nConfusion matrix of Logistic Regression optimized')
#     print(pd.DataFrame(confusion_matrix(y_test, y_pred),
#                  columns=['pred_neg', 'pred_pos'], index=['neg', 'pos']))
    return grid_search

In [66]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)
scaler = MinMaxScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

# transform X_train_sc to a higher degree
poly_features = PolynomialFeatures(degree=2)
X_train_poly = poly_features.fit_transform(X_train_sc)
X_train_poly_colnames = poly_features.get_feature_names(X.columns)

X_test_poly = poly_features.transform(X_test_sc)

clf = LogisticRegression(class_weight = 'balanced')
# clf.fit(X_train_sc, y_train)
#grid_search_clf_rocauc, y_pred_rocauc = grid_search_wrapper(clf, X_train_sc, y_train, X_test_sc, y_test, 'roc_auc')
grid_search_clf_rocauc = grid_search_wrapper(clf, X_train_poly, y_train, 'roc_auc')

Best params for roc_auc
{'C': 0.001, 'fit_intercept': True, 'penalty': 'l2'}


In [133]:
X_train

,age,admissionweight,admissionheight,bmi,gender_Female,ethnicity_African American,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,...,immunosuppression,cirrhosis,activetx,ima,midur,oobventday1,oobintubday1,diabetes,visitnumber,heartrate
patientunitstayid,,,,,,,,,,,,,,,,,,,,,
1055700,70,70.5,157.5,28.420257,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,68.186508
3237558,81,67.5,190.5,18.600037,0,0,0,1,0,0,...,0,0,1,0,0,1,1,0,1,79.018868
2723468,78,64.0,167.6,22.784104,0,0,0,1,0,0,...,0,0,1,0,0,1,1,0,1,73.627178
2264029,82,72.6,162.6,27.459684,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,99.789474
1603987,70,50.8,172.7,17.032525,1,0,0,1,0,0,...,0,0,1,0,0,1,1,0,1,89.993031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1688240,65,105.1,178.0,33.171317,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,83.542373
404042,69,92.0,165.1,33.751547,1,1,0,0,0,0,...,0,0,1,0,0,0,0,1,1,66.806324
2731140,82,53.6,160.0,20.937500,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,91.157343


In [77]:
# C=0.001, fit_intercept=True, penalty='l2'
clf_w_best_params = LogisticRegression(class_weight='balanced', C=0.001, fit_intercept=True, penalty='l2')


In [79]:
y_train_pred = clf_w_best_params.fit(X_train_poly, y_train)
y_train_probs = clf_w_best_params.predict_proba(X_train_poly)[:,1]
print('Training AUC with all features is ' + str(roc_auc_score(y_train, y_train_probs)))

Training AUC with all features is 0.7511785521064368


# What I had before

In [ ]:
# clf_w_best_params = LogisticRegression(C=0.01, fit_intercept=True, penalty='l2')

# #clf_w_best_params = LogisticRegression(class_weight={0: 0.005, 1: 0.995}, C=0.01, fit_intercept=False, penalty='l2')
# clf_w_best_params.fit(X_train_sc, y_train)
# file_name_pickle = 'model_' + now_to_str() + '.pickle'
# scaler_name_pickle = 'scaler_' + now_to_str() + '.pickle'
# pickle.dump(clf_w_best_params, open(file_name_pickle, 'wb'))
# pickle.dump(scaler, open(scaler_name_pickle, 'wb'))

# y_pred = clf_w_best_params.predict(X_test_sc)
# y_probs = clf_w_best_params.predict_proba(X_test_sc)[:,1]
# # print('accuracy is ' + str(accuracy_score(y_test, y_pred)))
# # print('precision is ' + str(precision_score(y_test, y_pred)))



# # print('F1 score is ' + str(f1_score(y_test, y_pred_rocauc)))
# print('AUC is ' + str(roc_auc_score(y_test,y_probs)))
# # print('Min prob is ' + str(y_probs.min()))
# # print('Max prob is ' + str(y_probs.max()))

In [ ]:
# y_probs.max()

In [ ]:
# %matplotlib qt


# noskill_probs = [0 for _ in range(len(y_test))]
# ns_fpr, ns_tpr, _ = roc_curve(y_test, noskill_probs)
# lr_fpr, lr_tpr, threshold_array = roc_curve(y_test, clf_w_best_params.predict_proba(X_test_sc)[:,1])

# y_pred_new = lr_tpr>0

# logisticRegr_auc = roc_auc_score(y_test, y_probs)
# print('Logistic: ROC AUC=%.3f' % (logisticRegr_auc))

# plt.figure()
# # plot the roc curve for the model
# plt.plot(ns_fpr, ns_tpr, linestyle='--')
# plt.plot(lr_fpr, lr_tpr, marker='.', label='Logistic Regr.')
# # axis labels
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# # # show the legend
# plt.legend()
# # # show the plot
# plt.show()

In [ ]:
# # Looking at ROC curve, a TPR of 0.88 and FPR of 0.48 seems reasonable because the prophylaxis may
# # cause bleeding, but VTE is deadly, too. Get the threshold at this level
# fpr_tpr_thresh = pd.DataFrame({'fpr': lr_fpr, 'tpr': lr_tpr, 'thresh': threshold_array})
# fpr_tpr_thresh = fpr_tpr_thresh[(fpr_tpr_thresh['tpr'] < 0.88) & (fpr_tpr_thresh['tpr'] > 0.87)]
# print(fpr_tpr_thresh)

In [ ]:
# Xt = X_test_sc
# yt = y_test

# THRESHOLD = [0.0017, 0.0018, 0.0019, 0.0020, 0.0021, 0.0022, 0.0023, 0.0024, 0.0025, 0.0026, 0.0027]
# for thresh in THRESHOLD:
#     print('-----------------')
#     print('THRESHOLD =  ' + str(thresh))
#     y_pred_thresh = np.where(clf_w_best_params.predict_proba(Xt)[:,1] > thresh, 1, 0)
#     print('F1: ' + str(f1_score(yt, y_pred_thresh)))
#     print('Precision: ' + str(precision_score(yt, y_pred_thresh)))
#     print('Recall: ' + str(recall_score(yt, y_pred_thresh)))
#     print(confusion_matrix(yt, y_pred_thresh, normalize='true'))

# Feature Weights

In [67]:
# Get feature weights and put into dataframe
mydict = {'feature': X_train_poly_colnames, 'coef': list(clf_w_best_params.coef_.reshape(-1,1).flatten())}
features_weights = pd.DataFrame(mydict)
features_weights = features_weights.assign(abs_weight=np.abs(features_weights['coef']))
features_weights.head()

,feature,coef,abs_weight
0,1,-0.000003,0.000003
1,age,0.025936,0.025936
2,admissionweight,0.029186,0.029186
3,admissionheight,0.002115,0.002115
4,bmi,0.029475,0.029475


In [68]:
features_weights = features_weights.sort_values(by='abs_weight', ascending=False)
features_weights.set_index('feature')

,coef,abs_weight
feature,,
heartrate,0.098791,0.098791
heartrate^2,0.070581,0.070581
activetx,0.064967,0.064967
oobintubday1^2,0.062797,0.062797
oobventday1,0.058392,0.058392
...,...,...
gender_Female aids,-0.000016,0.000016
aids cirrhosis,0.000012,0.000012
ethnicity_Hispanic ethnicity_Native American,-0.000009,0.000009


In [69]:
features_weights.shape

(496, 3)

In [70]:
features_weights.head(30)

,feature,coef,abs_weight
30,heartrate,0.098791,0.098791
495,heartrate^2,0.070581,0.070581
23,activetx,0.064967,0.064967
486,oobintubday1^2,0.062797,0.062797
26,oobventday1,0.058392,0.058392
27,oobintubday1,0.055709,0.055709
482,oobventday1 oobintubday1,0.045794,0.045794
481,oobventday1^2,0.043053,0.043053
393,hepaticfailure metastaticcancer,0.041213,0.041213
31,age^2,-0.034779,0.034779


In [71]:
fw = features_weights[['feature', 'abs_weight']]

In [72]:
fw.head(30)

,feature,abs_weight
30,heartrate,0.098791
495,heartrate^2,0.070581
23,activetx,0.064967
486,oobintubday1^2,0.062797
26,oobventday1,0.058392
27,oobintubday1,0.055709
482,oobventday1 oobintubday1,0.045794
481,oobventday1^2,0.043053
393,hepaticfailure metastaticcancer,0.041213
31,age^2,0.034779


In [76]:
%matplotlib qt
fw.plot.bar(x='feature', y='abs_weight')
plt.xticks([])
plt.xlabel('Polynomial Feature Index')
plt.ylabel('Abs. Value of Coefficient')
plt.tight_layout()
plt.legend().set_visible(False)

# Feature Selection

In [105]:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html
# https://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_with_cross_validation.html#sphx-glr-auto-examples-feature-selection-plot-rfe-with-cross-validation-py
from sklearn.feature_selection import RFECV
rfecv = RFECV(clf_w_best_params, step=1, cv=StratifiedKFold(n_splits=5), n_jobs=4, scoring='roc_auc')
rfecv.fit(X_train_poly, y_train)
print("Optimal number of features : %d" % rfecv.n_features_)



Optimal number of features : 15


In [118]:
pickle.dump(rfecv, open('../models/rfecv_logisticregr_poly' + now_to_str() + '.pickle', 'wb'))

In [119]:
# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("AUC")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.tight_layout()
plt.show()

In [117]:
rfecv.grid_scores_.max()

0.71374286880329

In [93]:
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.tight_layout()
plt.show()

In [121]:
X_train_poly.shape

(117013, 496)

In [131]:
X_train_poly_15feats = rfecv.transform(X_train_poly)
X_test_poly_15feats = rfecv.transform(X_test_poly)

In [130]:
print(sorted(zip(map(lambda x: round(x, 4), rfecv.ranking_), X_train_poly_colnames)))

[(1, 'activetx heartrate'), (1, 'activetx ima'), (1, 'activetx oobintubday1'), (1, 'admissionheight activetx'), (1, 'admissionweight activetx'), (1, 'age heartrate'), (1, 'eyes heartrate'), (1, 'heartrate'), (1, 'heartrate^2'), (1, 'ima'), (1, 'ima^2'), (1, 'motor eyes'), (1, 'motor heartrate'), (1, 'verbal oobventday1'), (1, 'verbal^2'), (2, 'age diabetes'), (3, 'verbal heartrate'), (4, 'verbal motor'), (5, 'age ethnicity_African American'), (6, 'ethnicity_Caucasian heartrate'), (7, 'ima oobventday1'), (8, 'admissionheight heartrate'), (9, 'immunosuppression^2'), (10, 'bmi activetx'), (11, 'ima oobintubday1'), (12, 'verbal oobintubday1'), (13, 'verbal'), (14, 'admissionweight'), (15, 'age eyes'), (16, 'ethnicity_Hispanic verbal'), (17, 'ethnicity_African American^2'), (18, 'ethnicity_Caucasian activetx'), (19, 'motor^2'), (20, 'gender_Female heartrate'), (21, 'gender_Female activetx'), (22, 'gender_Female verbal'), (23, 'motor ima'), (24, 'oobintubday1'), (25, 'admissionweight heartra

# Prediction With Most Important Features

In [ ]:
clf_w_best_params = LogisticRegression(class_weight='balanced', C=0.001, fit_intercept=True, penalty='l2')

#clf_w_best_params = LogisticRegression(class_weight={0: 0.005, 1: 0.995}, C=0.01, fit_intercept=False, penalty='l2')
clf_w_best_params.fit(X_train_poly_15feats, y_train)
# file_name_pickle = 'model_' + now_to_str() + '.pickle'
# scaler_name_pickle = 'scaler_' + now_to_str() + '.pickle'
# pickle.dump(clf_w_best_params, open(file_name_pickle, 'wb'))
# pickle.dump(scaler, open(scaler_name_pickle, 'wb'))

scaler = MinMaxScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)


model_and_scaler = {'model': clf_w_best_params, 'scaler': scaler}
pickle.dump(model_and_scaler, open('../models/model_scaler_' + now_to_str() + '.pickle', 'wb'))

y_pred = clf_w_best_params.predict(X_test_sc)
y_probs = clf_w_best_params.predict_proba(X_test_sc)[:,1]
# print('accuracy is ' + str(accuracy_score(y_test, y_pred)))
# print('precision is ' + str(precision_score(y_test, y_pred)))



# print('F1 score is ' + str(f1_score(y_test, y_pred_rocauc)))
print('AUC is ' + str(roc_auc_score(y_test,y_probs)))